## Figure 2 - Main text

### Load required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
from scipy.io import loadmat
import seaborn as sns
import matplotlib.ticker
import matplotlib.gridspec as gridspec

from multiprocessing import Pool
import multiprocessing as mp


### Load posteriors and PE results

In [ ]:
# Bayesian
pM1b = np.asarray(pd.read_csv("ParametersCSV/draws_ALL_Model1.csv"))
pM2b = np.asarray(pd.read_csv("ParametersCSV/draws_ALL_Model2.csv"))
pM3b = np.asarray(pd.read_csv("ParametersCSV/draws_ALL_Model3.csv"))

# Frequentist
pM1f = loadmat("ParametersCSV/BestPEM1.mat")
pM2f = loadmat("ParametersCSV/BestPEM2.mat")
pM3f = loadmat("ParametersCSV/BestPEM3.mat")


In [ ]:
# Bayesian
muB1 = np.mean(pM1b,0)
covB1 = np.cov(np.matrix.transpose(pM1b))

muB2 = np.mean(pM2b,0)
covB2 = np.cov(np.matrix.transpose(pM2b))

muB3 = np.mean(pM3b,0)
covB3 = np.cov(np.matrix.transpose(pM3b))

# Frequentist
muF1 = pM1f['bm1']['pe_results'][0,0]['fit'][0,0]['thetabest'][0,0].reshape(1,16)[0]
covF1 = pM1f['bm1']['pe_results'][0,0]['fit'][0,0]['g_var_cov_mat'][0,0]

muF2 = pM2f['bm2']['pe_results'][0,0]['fit'][0,0]['thetabest'][0,0].reshape(1,14)[0]
covF2 = pM2f['bm2']['pe_results'][0,0]['fit'][0,0]['g_var_cov_mat'][0,0]

muF3 = pM3f['bm3']['pe_results'][0,0]['fit'][0,0]['thetabest'][0,0].reshape(1,14)[0]
covF3 = pM3f['bm3']['pe_results'][0,0]['fit'][0,0]['g_var_cov_mat'][0,0]



### Definition of the Bhattacharyya distance and selection of the best and forst case

In [ ]:
def BhattacharyyaDist (mu1, mu2, sd1, sd2):
    
    t1 = (mu1-mu2)**2
    Em1 = (sd1**2)+(sd2**2)
    
    co1 = (sd1**2)/(sd2**2)
    co2 = (sd2**2)/(sd1**2)
    
    
    ft = (1/4)*(t1/Em1)
    st = (1/4)*(co1+co2+2)
    
    bhd = ft+0.25*np.log(st)
    
    return(float(bhd))

#### Model 1

In [ ]:
# Compute distribution distances
BDM1 = np.zeros(16)
for i in range(0,16):
    BDM1[i] = BhattacharyyaDist(muB1[i], muF1[i], np.sqrt(covB1[i,i]), np.sqrt(covF1[i,i]))
    
# Best and worst cases
BeM1 = list(BDM1).index(min(BDM1))
WoM1 = list(BDM1).index(max(BDM1))

In [ ]:
sdsM1 = np.zeros(16)
for i in range(0,16):
    sdsM1[i] = np.sqrt(covF1[i,i])

In [ ]:
WoM1 = list(sdsM1).index(max(sdsM1))

#### Model 2

In [ ]:
# Compute distribution distances
BDM2 = np.zeros(14)
for i in range(0,14):
    BDM2[i] = BhattacharyyaDist(muB2[i], muF2[i], np.sqrt(covB2[i,i]), np.sqrt(covF2[i,i]))
    
# Best and worst cases
BeM2 = list(BDM2).index(min(BDM2))
WoM2 = list(BDM2).index(max(BDM2))

In [ ]:
sdsM2 = np.zeros(14)
for i in range(0,14):
    sdsM2[i] = np.sqrt(covF2[i,i])
WoM2 = list(sdsM2).index(max(sdsM2))

#### Model 3

In [ ]:
# Compute distribution distances
BDM3 = np.zeros(14)
for i in range(0,14):
    BDM3[i] = BhattacharyyaDist(muB3[i], muF3[i], np.sqrt(covB3[i,i]), np.sqrt(covF3[i,i]))
    
# Best and worst cases
BeM3 = list(BDM3).index(min(BDM3))
WoM3 = list(BDM3).index(max(BDM3))

In [ ]:
sdsM3 = np.zeros(14)
for i in range(0,14):
    sdsM3[i] = np.sqrt(covF3[i,i])
WoM3 = list(sdsM3).index(max(sdsM3))

### Plot individual Distributions for comparison

#### Model 1

In [ ]:
M1Thetanams = ['k_in_IPTG ',
    'k_out_IPTG',
    'k_in_aTc  ',
    'k_out_aTc ',
    'kL_p_m0   ',
    'kL_p_m    ',
    'theta_T   ',
    'theta_aTc ',
    'n_aTc     ',
    'n_T       ',
    'kT_p_m0   ',
    'kT_p_m    ',
    'theta_L   ',
    'theta_IPTG',
    'n_IPTG    ',
    'n_L       ']

In [ ]:
# Model 1
fig = plt.figure(figsize=(5.7, 4.5), dpi = 250)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
gs = gridspec.GridSpec(1, 2, hspace = 1.5)

s = np.random.normal(muF1[BeM1], np.sqrt(covF1[BeM1,BeM1]), 1000)

ax1 = plt.subplot(gs[0, 0])
sns.distplot(pM1b[:,BeM1], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax1.tick_params(length=1.5)
ax1.set_ylabel('GFP (AU)')
ax1.set_xlabel('time (min)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel(M1Thetanams[BeM1])
ax1.set_title('Best')
ax1.legend()

s = np.random.normal(muF1[WoM1], np.sqrt(covF1[WoM1,WoM1]), 1000)
ax2 = plt.subplot(gs[0, 1])
sns.distplot(pM1b[:,WoM1], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax2.tick_params(length=1.5)
ax2.set_ylabel('GFP (AU)')
ax2.set_xlabel('time (min)')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_xlabel(M1Thetanams[WoM1])
ax2.set_title('Worst')
ax2.legend()

plt.show()

#### Model 2

In [ ]:
M23Thetanams = ['k_IPTG ',
    'k_aTc  ',
    'kL_p_m0   ',
    'kL_p_m    ',
    'theta_T   ',
    'theta_aTc ',
    'n_aTc     ',
    'n_T       ',
    'kT_p_m0   ',
    'kT_p_m    ',
    'theta_L   ',
    'theta_IPTG',
    'n_IPTG    ',
    'n_L       ']

In [ ]:
# Model 2
fig = plt.figure(figsize=(5.7, 4.5), dpi = 250)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
gs = gridspec.GridSpec(1, 2, hspace = 1.5)

s = np.random.normal(muF2[BeM2], np.sqrt(covF2[BeM2,BeM2]), 1000)

ax1 = plt.subplot(gs[0, 0])
sns.distplot(pM2b[:,BeM2], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax1.tick_params(length=1.5)
ax1.set_ylabel('GFP (AU)')
ax1.set_xlabel('time (min)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel(M23Thetanams[BeM2])
ax1.set_title('Best')
ax1.legend()

s = np.random.normal(muF2[WoM2], np.sqrt(covF2[WoM2,WoM2]), 1000)
ax2 = plt.subplot(gs[0, 1])
sns.distplot(pM2b[:,WoM2], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax2.tick_params(length=1.5)
ax2.set_ylabel('GFP (AU)')
ax2.set_xlabel('time (min)')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_xlabel(M23Thetanams[WoM2])
ax2.set_title('Worst')
ax2.legend()

plt.show()

#### Model 3

In [ ]:
# Model 3
fig = plt.figure(figsize=(5.7, 4.5), dpi = 250)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
gs = gridspec.GridSpec(1, 2, hspace = 1.5)

s = np.random.normal(muF3[BeM3], np.sqrt(covF3[BeM3,BeM3]), 1000)

ax1 = plt.subplot(gs[0, 0])
sns.distplot(pM3b[:,BeM3], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax1.tick_params(length=1.5)
ax1.set_ylabel('GFP (AU)')
ax1.set_xlabel('time (min)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel(M23Thetanams[BeM3])
ax1.set_title('Best')
ax1.legend()

s = np.random.normal(muF3[WoM3], np.sqrt(covF3[WoM3,WoM3]), 1000)
ax2 = plt.subplot(gs[0, 1])
sns.distplot(pM3b[:,WoM3], hist=False, rug=False, label = 'Bayesian');
sns.distplot(s, hist=False, rug=False, label = 'Frequentist');
ax2.tick_params(length=1.5)
ax2.set_ylabel('GFP (AU)')
ax2.set_xlabel('time (min)')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_xlabel(M23Thetanams[WoM3])
ax2.set_title('Worst')
ax2.legend()

plt.show()

#### Get distributions data

In [ ]:
# Model 1
dtM1 = {'BestBay': pM1b[:,BeM1] ,'WorstBay': pM1b[:,WoM1], 
      'BestFreq': np.random.normal(muF1[BeM1], np.sqrt(covF1[BeM1,BeM1]), 8000), 
      'WorstFrec': np.random.normal(muF1[WoM1], np.sqrt(covF1[WoM1,WoM1]), 8000)}
dfM1 = pd.DataFrame(dtM1)

# Model 2
dtM2 = {'BestBay': pM2b[:,BeM2] ,'WorstBay': pM2b[:,WoM2], 
      'BestFreq': np.random.normal(muF2[BeM2], np.sqrt(covF2[BeM2,BeM2]), 8000), 
      'WorstFrec': np.random.normal(muF2[WoM2], np.sqrt(covF2[WoM2,WoM2]), 8000)}
dfM2 = pd.DataFrame(dtM2)

# Model 3
dtM3 = {'BestBay': pM3b[:,BeM3] ,'WorstBay': pM3b[:,WoM3], 
      'BestFreq': np.random.normal(muF3[BeM3], np.sqrt(covF3[BeM3,BeM3]), 8000), 
      'WorstFrec': np.random.normal(muF1[WoM3], np.sqrt(covF3[WoM3,WoM3]), 8000)}
dfM3 = pd.DataFrame(dtM3)

In [ ]:
# Data
C4I = (pd.read_csv("Calibration_4_Events_Inputs.csv"))
C4D = (pd.read_csv("Calibration_4_Observables.csv"))
Inputs = [C4I]
Outputs = [C4D]


In [ ]:
parMin = [0.0036,0.0036,
          0.0089,0.0089,
          0.0027,0.8913,2.6738,0.8913,0,0,
          0.0056,0.0891,2.6738,0.0089,0,0]

In [ ]:
import pickle

In [ ]:
tmp1 = pickle.load(open('BayesianSimulations.pkl','rb'))
tmp2 = pickle.load(open('FrequentistSimulations.pkl','rb'))

#### Plot

In [ ]:
#fig = plt.figure(figsize=(5.7, 4.5), dpi = 250)
fig = plt.figure(figsize=(5.8,3.2), dpi = 250)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
gs = gridspec.GridSpec(8, 8, hspace = 1.5)

ax1 = plt.subplot(gs[2:4, 0:2])
# sns.jointplot(x=dfM1['BestBay'], y=dfM1['WorstBay'], kind='kde',color="skyblue")

ax1.scatter(dfM1['BestFreq'], dfM1['WorstFrec'], color = '#d9d9d9', marker='.', s=1, label='F')
ax1.scatter(dfM1['BestBay'], dfM1['WorstBay'], color='#525252', marker='.', s=1, label = 'B')
ax1.tick_params(length=1.5)
ax1.set_xlabel(M1Thetanams[BeM1])
ax1.set_ylabel(M1Thetanams[WoM1])
ax1.set_ylim([-100,300])
ax1.set_title('Model 1')
ax1.legend(framealpha = 0.0,bbox_to_anchor=(1.05, 1.0, 0.3, 0.2), loc='upper center')

ax2 = plt.subplot(gs[4:6,0:2])

ax2.scatter(dfM2['BestFreq'], dfM2['WorstFrec'], color = '#d9d9d9', marker='.', s=1, label='F')
ax2.scatter(dfM2['BestBay'], dfM2['WorstBay'], color='#525252', marker='.', s=1, label = 'B')
ax2.tick_params(length=1.5)
ax2.set_xlabel(M23Thetanams[BeM2])
ax2.set_ylabel(M23Thetanams[WoM2])
ax2.set_ylim([-100,300])
ax2.set_title('Model 2')
#ax2.legend()

ax3 = plt.subplot(gs[6:8,0:2])

ax3.scatter(dfM3['BestFreq'], dfM3['WorstFrec'], color = '#d9d9d9', marker='.', s=1, label='F')
ax3.scatter(dfM3['BestBay'], dfM3['WorstBay'], color='#525252', marker='.', s=1, label = 'B')
ax3.tick_params(length=1.5)
ax3.set_xlabel(M23Thetanams[BeM3])
ax3.set_ylabel(M23Thetanams[WoM3])
ax3.set_ylim([-100,300])
ax3.set_title('Model 3')
#ax3.legend()

ax4 = plt.subplot(gs[0:3, 3:8])
exp = 0
sp = list(np.around(np.append(np.asarray(Inputs[exp]['Switchingtimes']), Inputs[exp]['FinalTime'][0])).astype(int))
ts =  np.around(np.asarray(Outputs[0]['timeGFP'])).astype(int)
t = ts
    

#lines1 = ax4.plot(t, GFP_M1f,color='#039a00')
lines1 = ax4.plot(t, tmp2['Sims'][1],color='#ccece6',linewidth=0.5)
#'#99d8c9'
ax4.plot(0,0,color='#ccece6', label = '$F$')
#lines2 = ax4.plot(t, GFP_M1,color='#10fb00')
lines2 = ax4.plot(t, tmp1['Sims'][1], color='#41ae76',linewidth=0.5)
ax4.plot(0,0,color='#41ae76', label = '$B$')

ax4.errorbar(Outputs[0]['timeGFP'], Outputs[0]['GFPmean'], yerr= Outputs[0]['GFPstd'],fmt='.',markersize = 2,color='grey',elinewidth=0.3,alpha = 0.3)

ax4.tick_params(length=1.5)
ax4.set_ylabel('GFP (AU)')
ax4.set_xlabel('time (min)')
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.set_xlabel('')
ax4.set_title('Calibration 4')
plt.setp(ax4.get_xticklabels(), visible=False)
ax4.legend(loc=2)

    
ax5 = plt.subplot(gs[3:6, 3:8])

#lines1 = ax5.plot(t, RFP_M1f,color='#f44b4b')
lines1 = ax5.plot(t, tmp2['Sims'][0],color='#fee0d2',linewidth=0.5)
#fcbba1'
ax5.plot(0,0,color='#fee0d2', label = '$F$')

#lines2 = ax5.plot(t, RFP_M1, color='#d61111')
lines2 = ax5.plot(t, tmp1['Sims'][0], color='#ef3b2c',linewidth = 0.5)
ax5.plot(0,0,color='#ef3b2c', label = '$B$')

ax5.errorbar(Outputs[0]['timeRFP'], Outputs[0]['RFPmean'], yerr= Outputs[0]['RFPstd'],fmt='.',markersize = 2,color='grey',elinewidth=0.3,alpha = 0.3)


ax5.tick_params(length=1.5)
ax5.set_ylabel('RFP (AU)')
ax5.set_xlabel('time (min)')
ax5.spines['top'].set_visible(False)
ax5.spines['right'].set_visible(False)
ax5.set_xlabel('')
ax5.set_title('Calibration 4')
plt.setp(ax5.get_xticklabels(), visible=False)
ax5.legend(loc=1)


ax6 = plt.subplot(gs[6, 3:8])
ax6.step(sp, [(Inputs[exp]['IPTG'][0])]+ list(Inputs[exp]['IPTG']), 'cyan')
ax6.set_xticklabels('')
ax6.set_ylabel('IPTG \n (mM)')
ax6.tick_params(length=1.5)
ax6.spines['right'].set_visible(False)
ax6.spines['top'].set_visible(False)
    
ax7 = plt.subplot(gs[7, 3:8])
ax7.step(sp, [(Inputs[exp]['aTc'][0])]+ list(Inputs[exp]['aTc']), 'violet')
ax7.set_xlabel('time (min)')
ax7.set_ylabel('aTc \n (ng/ml)')
ax7.tick_params(length=1.5)
ax7.spines['right'].set_visible(False)
ax7.spines['top'].set_visible(False)


plt.show()